# 03 — Insights and Predictive Sanity Check

**Objective.** This notebook reproduces the insight queries and summarizes a predictive sanity check using BigQuery ML (BQML).

**Tables and Models.**
- Curated data: `PROJECT_ID.DATASET.curated_usage`
- BQML model: `PROJECT_ID.DATASET.lusage_linreg`

In [ ]:
# Configuration
PROJECT_ID = "data-engineering-ecometricx"
DATASET = "energy_analytics"
LOCATION = "EU"

# !pip -q install --upgrade pandas pandas-gbq google-cloud-bigquery

import pandas as pd
from pandas_gbq import read_gbq
import matplotlib.pyplot as plt

CUR = f"`{PROJECT_ID}.energy_analytics.curated_usage`".format(PROJECT_ID=PROJECT_ID)
STG = f"`{PROJECT_ID}.energy_analytics.stg_usage`".format(PROJECT_ID=PROJECT_ID)
MDL = f"`{PROJECT_ID}.energy_analytics.lusage_linreg`".format(PROJECT_ID=PROJECT_ID)

## 1. Pre versus post summary

In [ ]:
q = (
    "WITH pre AS ( "
    "SELECT 'pre' AS stage, "
    "APPROX_QUANTILES(lusage, 100)[OFFSET(50)] AS median, "
    "AVG(lusage) AS avg, COUNT(*) AS n, COUNTIF(lusage IS NULL) AS miss "
    f"FROM {STG} "
    "), post AS ( "
    "SELECT 'post' AS stage, "
    "APPROX_QUANTILES(lusage, 100)[OFFSET(50)] AS median, "
    "AVG(lusage) AS avg, COUNT(*) AS n, COUNTIF(was_imputed) AS imputed_n "
    f"FROM {CUR} "
    ") "
    "SELECT * FROM pre UNION ALL SELECT * FROM post"
)
prepost = read_gbq(q, project_id=PROJECT_ID, location=LOCATION)
prepost

## 2. Monthly averages (post-curated)

In [ ]:
q = f"SELECT month, AVG(lusage) AS avg_lusage, COUNT(*) n FROM {CUR} GROUP BY month ORDER BY month"
season = read_gbq(q, project_id=PROJECT_ID, location=LOCATION)
plt.figure()
plt.plot(season['month'], season['avg_lusage'], marker='o')
plt.title('Average log usage by month (curated)')
plt.xlabel('month'); plt.ylabel('average lusage')
plt.xticks([4,5,6,7,8])
plt.show()
season

## 3. Lag correlations (post-curated)

In [ ]:
q = f"SELECT CORR(lusage,lusage1) AS corr_l1, CORR(lusage,lusage2) AS corr_l2, CORR(lusage,lusage3) AS corr_l3 FROM {CUR}"
cor = read_gbq(q, project_id=PROJECT_ID, location=LOCATION)
cor

## 4. Predictive sanity check (BQML evaluation)

In [ ]:
try:
    eval_df = read_gbq(f"SELECT * FROM ML.EVALUATE(MODEL {MDL})", project_id=PROJECT_ID, location=LOCATION)
    eval_df
    plt.figure()
    plt.bar(['MAE','MedAE'], [float(eval_df['mean_absolute_error'][0]), float(eval_df['median_absolute_error'][0])])
    plt.title('Model error summary (BQML)')
    plt.ylabel('error (log kWh)')
    plt.show()
except Exception as e:
    print("Model evaluation not available or query failed:", e)

## 5. Interpretation and limitations

- The curated series replicates the staging distribution because `lusage` contained few or no missing values; baseline imputation had minimal effect.
- Lag correlations are strong, consistent with persistence in household electricity usage.
- The BQML linear regression may exhibit overfitting without additional regularization and higher-quality exogenous features (e.g., weather indices beyond `mozip`). Out-of-time validation is recommended.